<a href="https://colab.research.google.com/github/IAMJASURBEK/AIapplicationSystem/blob/main/week7/week7_session2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1-2

In [1]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
# Save datagenerators as file to colab working directory
# If you are using GitHub, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/NVDLI/LDL/main/pt_framework/utilities.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import
with open('utilities.py', 'w') as f:
  f.write(r.text)

## Step 1-3

In [3]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

import numpy as np

from utilities import train_model

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 128
BATCH_SIZE = 32

In [5]:
# Load training dataset into a single batch to compute mean and stddev.
transform = transforms.Compose([transforms.ToTensor()])
trainset = CIFAR10(root='./pt_data', train = True, download = True, transform = transform)
trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
data = next(iter(trainloader))
mean = data[0].mean()
stddev = data[0].std()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./pt_data/cifar-10-python.tar.gz to ./pt_data


## Step 1-4

In [6]:
# Load and standardize training and test dataset.
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(mean, stddev)
                                ])

trainset = CIFAR10(root='./pt_data', train = True, download = True, transform = transform)
testset = CIFAR10(root='./pt_data', train = True, download = True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Model with 2 convolutional and 1 fully-connected layer.
model = nn.Sequential(
    nn.Conv2d(3, 64, 5, stride = 2, padding = 2), # Output is 64x16x16
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, stride = 2, padding = 1), # Output is 64x8x8
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(64 * 8 * 8, 10)
)

## Step 1-5

In [8]:
# Retrieve layers for custom weight initialization.
layers = next(model.modules())
conv_layer0 = layers[0]
conv_layer1 = layers[2]
output_layer = layers[5]

# Kaiming (He) initialization.
nn.init.kaiming_normal_(conv_layer0.weight)
nn.init.constant_(conv_layer0.bias, 0.0)
nn.init.kaiming_normal_(conv_layer1.weight)
nn.init.constant_(conv_layer1.bias, 0.0)

# Xavier (Glorot) initialization.
nn.init.xavier_uniform_(output_layer.weight)
nn.init.constant_(output_layer.bias, 0.0)

# Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()

# Train the model
train_model(model, device, EPOCHS, BATCH_SIZE, trainset, testset, optimizer, loss_function, 'acc')

Epoch 1/128 loss: 1.3883 - acc: 0.5093 - val_loss: 1.0574 - val_acc: 0.6320
Epoch 2/128 loss: 1.0627 - acc: 0.6298 - val_loss: 0.8881 - val_acc: 0.6911
Epoch 3/128 loss: 0.9198 - acc: 0.6793 - val_loss: 0.7592 - val_acc: 0.7363
Epoch 4/128 loss: 0.8169 - acc: 0.7158 - val_loss: 0.6579 - val_acc: 0.7752
Epoch 5/128 loss: 0.7356 - acc: 0.7421 - val_loss: 0.6065 - val_acc: 0.7936
Epoch 6/128 loss: 0.6666 - acc: 0.7661 - val_loss: 0.5494 - val_acc: 0.8140
Epoch 7/128 loss: 0.6075 - acc: 0.7869 - val_loss: 0.4598 - val_acc: 0.8457
Epoch 8/128 loss: 0.5514 - acc: 0.8048 - val_loss: 0.4147 - val_acc: 0.8578
Epoch 9/128 loss: 0.5091 - acc: 0.8206 - val_loss: 0.3945 - val_acc: 0.8658
Epoch 10/128 loss: 0.4658 - acc: 0.8344 - val_loss: 0.3548 - val_acc: 0.8775
Epoch 11/128 loss: 0.4299 - acc: 0.8504 - val_loss: 0.3171 - val_acc: 0.8902
Epoch 12/128 loss: 0.3964 - acc: 0.8565 - val_loss: 0.3178 - val_acc: 0.8887
Epoch 13/128 loss: 0.3721 - acc: 0.8664 - val_loss: 0.2665 - val_acc: 0.9072
Epoch 14

[0.9749080294305822, 0.9788267754318618]